In [1]:
import numpy as np 
import pandas as pd

In [2]:
dataset=pd.read_csv('Groceries.csv',header=None)

In [5]:
pd.pandas.set_option('display.max_column',None)

In [6]:
dataset.shape

NameError: name 'dataset' is not defined

In [11]:
#Adding all transactions into list of lists..
groceries=[]
for i in range(0,3000):
    groceries.append([str(dataset.values[i,j]) for j in range(0,10)])
    
    

In [15]:
groceries[4:5]

[['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan',
  'nan']]

In [16]:
#Training Apriori on the dataset
from apyori import apriori
association_rules=apriori(groceries,min_support=0.0025,min_confidence=0.20,min_lift=3,min_length=2)

In [17]:
results=list(association_rules)

In [18]:
results[:1]

[RelationRecord(items=frozenset({'margarine', 'baking powder'}), support=0.004, ordered_statistics=[OrderedStatistic(items_base=frozenset({'baking powder'}), items_add=frozenset({'margarine'}), confidence=0.25531914893617025, lift=4.453240969816923)])]

In [19]:
lift=[]
association_items=[]
for i in range(0,len(results)):
    lift.append(results[:len(results)][i][2][0][3])
    association_items.append(list(results[:len(results)][i][0]))

In [20]:
#Putting it all together in a dataframe
final_groceries=pd.DataFrame([association_items,lift]).T
final_groceries.columns=['Association Items','Lift']

In [21]:
#TOp 10 highest lift scores
final_groceries.sort_values('Lift',ascending=False).head(10)

,Association Items,Lift
267,"[soda, yogurt, sausage, bottled water]",15.7895
325,"[dessert, pip fruit, yogurt, other vegetables]",14.4385
290,"[cream cheese , whole milk, domestic eggs, cit...",14.026
328,"[dessert, yogurt, other vegetables, whipped/so...",13.3185
288,"[other vegetables, chicken, tropical fruit, ro...",12.2727
320,"[curd, pip fruit, yogurt, tropical fruit]",11.2202
326,"[dessert, yogurt, sausage, other vegetables]",11.0294
243,"[whole milk, processed cheese, white bread]",10.9091
393,"[soda, canned beer, shopping bags, rolls/buns,...",10.4621
327,"[dessert, yogurt, tropical fruit, other vegeta...",10.2302


In [22]:
#From All this we can infer that "Soda , Yogurt , Sausage and Bottled Water " are the most common combined items.

In [23]:
#Eclat Implementation
#getting the products bought by customers...

groceriess=[]
for i in range(0,len(groceries)):
    groceriess.extend(groceries[i])
    
#Finding Unique items from differennt transactions and removing Nan
uniqueItems=list(set(groceriess))
uniqueItems.remove('nan')

In [24]:
# Creating Combinations with items-pairs....
pairs=[]
for j in range(0,len(uniqueItems)):
    k=1;
    while(k<len(uniqueItems)):
        try:
            pairs.append([uniqueItems[j],uniqueItems[j+k]])
        except IndexError:
            pass
        k=k+1;
            

In [28]:
uniqueItems[:6] #Sample Result after removing nans

['cereals',
 'sausage',
 'female sanitary products',
 'mayonnaise',
 'syrup',
 'rice']

In [31]:
#Calculating Score for each Item-pairs...
score=[]
for i in pairs:
    cond=[]
    for item in i:
        cond.append('("%s") in s' %item)
    x = ('[s for s in groceries if ' + ' and '.join(cond) + ']')    
    score.append(len(eval(x))/3000.0)

In [32]:
Eclat_results = pd.DataFrame([pairs, score]).T
Eclat_results.columns = ['Pairs', 'Score']

In [33]:
Eclat_results.sort_values('Score',ascending=False).head(10)

,Pairs,Score
3805,"[whole milk, other vegetables]",0.072
3826,"[whole milk, rolls/buns]",0.058
3873,"[whole milk, yogurt]",0.0563333
3836,"[whole milk, root vegetables]",0.048
4649,"[other vegetables, root vegetables]",0.0453333
4686,"[other vegetables, yogurt]",0.045
5667,"[soda, rolls/buns]",0.0423333
4639,"[other vegetables, rolls/buns]",0.041
3237,"[tropical fruit, whole milk]",0.0403333
2065,"[citrus fruit, whole milk]",0.0346667
